# BigData_SistemasRecomendacion_eq13

**Tecnológico de Monterrey**
**Maestría en Inteligencia Artificial Aplicada (MNA)**

**Materia: Big Data**

## Proyecto de Big Data sobre Sistemas de Recomendación.

Miembros del equipo:

Hansel Zapiain Rodriguez (A00469031)

Miguel Guillermo Galindo Orozco (A01793695)

Francisco José Arellano Montes (A01794283)


# Avance de proyecto 2: Problema de arranque en frio de filtrado basado en contenido (Content-Based Filtering)

# Importar Librerías

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from scipy.sparse import csr_matrix
#import sklearn
from sklearn.neighbors import NearestNeighbors

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
%matplotlib inline

El set de datos a usar será usando las reseñas por usuarios basados en productos cosméticos, llamaremos el set de datos usando la librería de data sets de huggingface.

In [ ]:
dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_All_Beauty", trust_remote_code=True)
print(dataset["full"][0])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating full split: 0 examples [00:00, ? examples/s]

{'rating': 5.0, 'title': 'Such a lovely scent but not overpowering.', 'text': "This spray is really nice. It smells really good, goes on really fine, and does the trick. I will say it feels like you need a lot of it though to get the texture I want. I have a lot of hair, medium thickness. I am comparing to other brands with yucky chemicals so I'm gonna stick with this. Try it!", 'images': [], 'asin': 'B00YQ6X8EO', 'parent_asin': 'B00YQ6X8EO', 'user_id': 'AGKHLEW2SOWHNMFQIJGBECAF7INQ', 'timestamp': 1588687728923, 'helpful_vote': 0, 'verified_purchase': True}


In [ ]:
dataset

DatasetDict({
    full: Dataset({
        features: ['rating', 'title', 'text', 'images', 'asin', 'parent_asin', 'user_id', 'timestamp', 'helpful_vote', 'verified_purchase'],
        num_rows: 701528
    })
})

In [ ]:
type(dataset)

datasets.dataset_dict.DatasetDict

In [ ]:
df = pd.DataFrame.from_dict(dataset["full"])

Obtención de los nombres de las columnas del set de datos.

In [ ]:
for i in df.columns:
    print(i)

rating
title
text
images
asin
parent_asin
user_id
timestamp
helpful_vote
verified_purchase


Juntaremos las columnas mas relevantes, este será el set de datos que usaremos para nuestro modelo y el análisis exploratorio de datos.

In [ ]:
df = df[['asin','parent_asin','user_id','rating','helpful_vote','verified_purchase']]
df.head(10)

,asin,parent_asin,user_id,rating,helpful_vote,verified_purchase
0,B00YQ6X8EO,B00YQ6X8EO,AGKHLEW2SOWHNMFQIJGBECAF7INQ,5.0,0,True
1,B081TJ8YS3,B081TJ8YS3,AGKHLEW2SOWHNMFQIJGBECAF7INQ,4.0,1,True
2,B07PNNCSP9,B097R46CSY,AE74DYR3QUGVPZJ3P7RFWBGIX7XQ,5.0,2,True
3,B09JS339BZ,B09JS339BZ,AFQLNQNQYFWQZPJQZS6V3NZU4QBQ,1.0,0,True
4,B08BZ63GMJ,B08BZ63GMJ,AFQLNQNQYFWQZPJQZS6V3NZU4QBQ,5.0,0,True
5,B00R8DXL44,B00R8DXL44,AGMJ3EMDVL6OWBJF7CA5RGJLXN5A,4.0,0,True
6,B099DRHW5V,B099DRHW5V,AHREXOGQPZDA6354MHH4ETSF3MCQ,5.0,0,True
7,B088SZDGXG,B08BBQ29N5,AEYORY2AVPMCPDV57CE337YU5LXA,3.0,0,True
8,B08P2DZB4X,B08P2DZB4X,AFSKPY37N3C43SOI5IEXEK5JSIYA,5.0,0,False
9,B086QY6T7N,B086QY6T7N,AFSKPY37N3C43SOI5IEXEK5JSIYA,5.0,0,False


# Exploratory Data Analysis (EDA)

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df.info()

print("\n")
print(df['asin'].isna().sum())
print(df['parent_asin'].isna().sum())
print(df['user_id'].isna().sum())
print(df['rating'].isna().sum())
print(df['helpful_vote'].isna().sum())
print(df['verified_purchase'].isna().sum())

Se puede observar que no existen datos nulo, lo que significa que no se requiere ningún tipo de imputación de datos.

In [ ]:
print("El Data Set a estudiar cuenta con " + str(len(df)) + " registros de rating de productos adquiridos en Amazon")

In [ ]:
#df['parent_asin'].nunique()
print("Son " + str(df['parent_asin'].nunique()) + " productos distintos (parent_asin)")

In [ ]:
print("De  " + str(df['user_id'].nunique()) + " usuarios distintos que dejaron su evaluación")

Veamos los productos que más veces fue evaludado (con más reseñas)

In [ ]:
df.groupby('parent_asin')['rating'].count().sort_values(ascending=False).head(10)

Igualmente, revisaremos los productos con reseñas más bajas

In [ ]:
df.groupby('parent_asin')['rating'].count().sort_values(ascending=True).head(10)

Observamos ahora cuál es el rating promedio de los productos con más evaluaciones

In [ ]:
ratings = pd.DataFrame(df.groupby('parent_asin')['rating'].mean())
ratings['num of ratings'] = pd.DataFrame(df.groupby('parent_asin')['rating'].count())
ratings['helpful_vote'] = pd.DataFrame(df.groupby('parent_asin')['helpful_vote'].count())
#ratings.head()
ratings.sort_values(by='num of ratings', ascending=False).head(100)

Cuántos productos existen con más de 300 evaluaciones

In [ ]:
print("Existen " + str(len(ratings[ratings['num of ratings']>300])) + " productos con al menos 300 evaluaciones")


De igual manera, queremos ver los productos con menos evaluaciones

In [ ]:
print("Existen " + str(len(ratings[ratings['num of ratings']<300])) + " productos con al menos 300 evaluaciones")

Ahora observaremos los outliers en nuestro set de datos

In [ ]:
plt.figure(figsize=(10,4))
sns.boxplot(x=df['parent_asin'])

In [ ]:
plt.figure(figsize=(10,4))
sns.boxplot(x=df['rating'])

In [ ]:
plt.figure(figsize=(10,4))
sns.boxplot(x=ratings['num of ratings'])

In [ ]:
plt.figure(figsize=(10,4))
sns.boxplot(x=ratings['rating'])

¿Cuántos productos se tienen por número de evaluaciones (al menos 300)?

In [ ]:
plt.figure(figsize=(10,4))
ratings[ratings['num of ratings']>300]['num of ratings'].hist(bins=70)

De lo anterior observamos que los productos entre 300 y 500 evaluaciones se concentran en 5 o 25 productos distintos

In [ ]:
plt.figure(figsize=(10,4))
ratings[ratings['num of ratings']<300]['num of ratings'].hist(bins=70)

Por el otro lado podemos observar, estando completamente en el otro extremo con muchos productos teniendo menos de 50 evaluaciones

La distribución de los ratings observados se ve de la siguiente forma

In [ ]:
plt.figure(figsize=(10,4))
ratings['rating'].hist(bins=70)

y para aquellos que tienen más de 300 evaluaciones se ve así:

In [ ]:
plt.figure(figsize=(10,4))
ratings[ratings['num of ratings']>300]['rating'].hist(bins=70)

Se observa que en general la mayoría de los productos con más de 300 evaluaciones tienen ratings por arriba de 3 / 3.5 de calificacación

Y para los que tienen menos de 300 evaluaciones se ve así:

In [ ]:
plt.figure(figsize=(10,4))
ratings[ratings['num of ratings']<300]['rating'].hist(bins=70)

Una distribución menos sesgada y concentrada en valores enteros, la mayoría fluctuando entre una calificación de 3.5 para abajo

In [ ]:
sns.jointplot(x='rating',y='num of ratings',data=ratings,alpha=0.5)

Con el gráfico anterior observamos una correlación fuerte entre items con más calificaciones tieneden a tener en promedio calificaciones más altas

In [ ]:
plt.figure(figsize = (16,5))
num_corr_matrix = ratings.corr()
ax = sns.heatmap(num_corr_matrix, annot=True)     # Sample figsize in inches

Podremos también los datos usando una matriz de correlación, podemos ver que de manera general, la matriz de correlación los votos y las evaluaciones parecen estar prácticamente de la mano.

# Muestreo

Debido al gran tamaño de la muestra y poder de procesamiento, decidimos hacer un muestro aleatorio sobre el 5% del total de la muestra.

De lo contrario, da un error por uso de RAM.

In [ ]:
df_sample = df.sample(frac=0.05, replace=False, random_state=1234)

In [ ]:
print(" El muestro aleatorio cuenta con " + str(len(df_sample)) + " registros")

 El muestro aleatorio cuenta con 35076 registros


In [ ]:
df_sample.head()

,asin,parent_asin,user_id,rating,helpful_vote,verified_purchase
169932,B07H7C4SJK,B07H7C4SJK,AFWUQ2CT6NCFBTTNI3OCWDB3CRYQ,5.0,2,True
117265,B07T9GXWLT,B07T9GXWLT,AGI5JHRJ2PSK5BURGV762KIG2Y5A,5.0,0,True
30312,B073R5WZ4F,B073R5WZ4F,AEDSDRACFPPSWJTYBRFPMHUMHWDQ,5.0,0,True
335686,B071S98JST,B071S98JST,AEHQ7LKVLDNZOYSMFQFJU6DORK3Q,4.0,0,True
101796,B077BZCPSK,B077BZCPSK,AHTCO6WZEFV2UURBBZCLOKLSA64A,3.0,0,True


# Sistema de Recomendación

El `filtrado colaborativo` se basa en las interacciones del usuario con los items (user-item) dentro de una matriz de utilidad. El problema reside en que los nuevos usuarios o los elementos que no cuenten con interacciones registradas van a ser ignoradas por el sistema de recomendacion. A esto se le conoce como el problema de "Arranque en frio". El filtrado basado en contenido es una manera de atacar este problema al generar recomendaciones en interacciones usuario-item.

Conteo de todas las evaluaciones totales

In [ ]:
from collections import Counter

In [ ]:
contador_ratings = Counter(df_sample['rating'])
contador_ratings

Counter({5.0: 21016, 4.0: 4024, 3.0: 2855, 2.0: 2158, 1.0: 5023})

Preparación de los datos teniendo columnas las calificaciones y las filas los productos.

In [ ]:
# converting to binary data
df_one = pd.get_dummies(df_sample['rating'], dtype=float)
df_one = df_one.reset_index()
df_one = df_one.drop(['index'], axis=1)
df_one.head()

,1.0,2.0,3.0,4.0,5.0
0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,0.0


In [ ]:
#users_items_pivot_matrix_df = df_sample.pivot(columns='asin',
#                                              values='rating').fillna(0)

Para construir este sistema de recomendacion item-item utilizaremos una metrica llamada similitud coseno `cosine similarity`.

La similitud Coseno se basa en el coseno del angulo entre dos vectores (c.e., $A$ y $B$). Entre mas pequeño es el angulo, mayor es la similitud entre los vectores, y se calcula de la siguiente forma:

$$\cos(\theta) = \frac{A\cdot B}{||A|| ||B||}$$

En este ejercicio, utilizaremos la funcion de scikit-learn de cosine_similarity para generar la matriz de similitud de tipo $(n_{\text{}}, n_{\text{}})$. Esta matriz nos permitira extraer productos que sean parecidas al producto de interes.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(df_one, df_one)
print(f"Las dimensiones de similaridad coseno de las caracteristicas de nuestra matriz de similitud son: {cosine_sim.shape}")

Las dimensiones de similaridad coseno de las caracteristicas de nuestra matriz de similitud son: (35076, 35076)


Desarrollando una funcion que encuentre peliculas similares

In [ ]:
!pip install fuzzywuzzy

In [ ]:
from fuzzywuzzy import process

def encuentra_prod_cercana(ev):
    all_ev = df_sample['asin'].tolist()
    closest_match = process.extractOne(ev, all_ev)
    return closest_match[0]

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
product = encuentra_prod_cercana('B07H7C4')
print(f"El producto es: {product}:")

El producto es: B07H7C4SJK:


Ahora que ya no nos preocupamos por escribir mal el producto, lo que nos interesa es encontrar recomendaciones relevantes a partir del producto que deseamos.

Deseamos encontrar el indice usando la matriz de similitud coseno.

In [ ]:
product_idx = dict(zip(df_sample['asin'], list(df_one.index)))
idx = product_idx[product]
print(f"El indice de la pelicula {product} en la matriz product_idx es: {idx}")

El indice de la pelicula B07H7C4SJK en la matriz product_idx es: 0


Al final obtenemos un arreglo de índices que representa el top 10 de recomendaciones a partir de producto elegido

In [ ]:
numero_recomendaciones=10
sim_scores = list(enumerate(cosine_sim[idx]))
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
sim_scores = sim_scores[1:(numero_recomendaciones+1)]
similar_prods = [i[0] for i in sim_scores]

In [ ]:
print(f"Por que miraste: {product}, te pueden interesar las siguientes {numero_recomendaciones}:")
df_sample['asin'].iloc[similar_prods]

Por que miraste: B07H7C4SJK, te pueden interesar las siguientes 10:


117265    B07T9GXWLT
30312     B073R5WZ4F
661048    B018C0AXU6
285302    B01719IZOU
630735    B09B33KQ9Z
370770    B06VWPZN56
152407    B007IAE5WY
576584    B00415EALU
228576    B0171UQIUW
558455    B074JYLYLZ
Name: asin, dtype: object